In [1]:
import os
import random   
import pandas as pd
# from underthesea import pos_tag
list1 = "qwertyuiopasdfghjklzxcvbnmư"
list2 = "`1234567890-=qwertyuiop[]\\asdfghjkl;'zxcvbnm,./"
dict1 = {'à':'aw', 'á':'as', 'ả':'ar', 'ã':'ax', 'ạ':'aj', 'ă':'aw', 'ằ':'afw', 'ắ':'asw', 'ẳ':'arw', 'ẵ':'axw', 'ặ':'ajw', 'â':'aa', 'ầ':'aaf', 'ấ':'aas', 'ẩ':'aar', 'ẫ':'aax', 'ậ':'aaj', 'đ':'dd', 'è':'ef', 'é':'es', 'ẻ':'er', 'ẽ':'ex', 'ẹ':'ej', 'ê':'ee', 'ề':'eef', 'ế':'ees', 'ể':'eer', 'ễ':'eex', 'ệ':'eej', 'ì':'if', 'í':'is', 'ỉ':'ir', 'ĩ':'ix', 'ị':'ij', 'ò':'of', 'ó':'os', 'ỏ':'or', 'õ':'ox', 'ọ':'oj', 'ô':'oo', 'ồ':'oof', 'ố':'oos', 'ổ':'oor', 'ỗ':'oox', 'ộ':'ooj', 'ơ':'ow', 'ờ':'owf', 'ớ':'ows', 'ở':'owr', 'ỡ':'owx', 'ợ':'owj', 'ù':'uf', 'ú':'us', 'ủ':'ur', 'ũ':'ux', 'ụ':'uj', 'ư':'uw', 'ừ':'uwf', 'ứ':'uws', 'ử':'uwr', 'ữ':'uwx', 'ự':'uwj', 'ỳ':'yf', 'ý':'ys', 'ỷ':'yr', 'ỹ':'yx', 'ỵ':'yj'}
def create_Error(sent, pivot):
    # kiểm tra pos tag của word trong câu, không tạo lỗi trên các word có tag 'Np"
#     sent_postag = pos_tag(sent)
    tokens = [x for x in sent.split()]
    out = []
    count = 0
    for i in range(len(sent.split())):
        token = sent.split()[i]
        # nếu là dấu hoặc các từ đặc biệt thì giữ nguyên, không tạo lỗi
        if not token.isalpha():
            out.append(0)
            continue
#         # không tạo lỗi trên các word có tag 'Np"
#         if sent_postag[i][1] == 'Np':
#             out.append(0)
#             continue
        prob = random.random()
        # nếu word trong 1 câu có xác suất nhỏ hơn pivot thì tiến hành tạo lỗi trên nó
        # make sure that number of error not large than 1/2 total number of tokens
        if prob < pivot and count <= len(tokens) // 2:
            count = count + 1
            prob = prob/pivot
                # đổi chỗ vị trí 2 char
            if prob < 0.2:
                modified_word = list(token)
                word_length = len(modified_word)
                if word_length - 2 <= 0:
                  out.append(0)
                  continue
                modified_index = random.randint(0, word_length - 2)
                modified_word[modified_index], modified_word[modified_index + 1] = modified_word[modified_index + 1], modified_word[modified_index]
                tokens[i] = ''.join(modified_word)
                out.append(1)
                # thêm 1 char vào word
            elif prob < 0.4:
                modified_word = list(token)
                word_length = len(modified_word)
                modified_index = random.randint(0, word_length - 1)
                y = random.randrange(len(list1))
                tokens[i] = ''.join(modified_word[:modified_index]) + list1[y] + ''.join(modified_word[modified_index:])
                out.append(1)
                # xóa 1 char khỏi word
            elif prob < 0.6:
                modified_word = list(token)
                word_length = len(modified_word)
                modified_index = random.randint(0, word_length - 1)
                tokens[i] =  ''.join(modified_word[:modified_index ] + modified_word[modified_index + 1:])
                out.append(1)
                # thay 1 char bằng 1 char khác
            elif prob < 0.8:
                modified_word = list(token)
                word_length = len(modified_word)
                modified_index = random.randint(0, word_length - 1)
                modified_word[modified_index] = random.choice(list2)
                tokens[i] = ''.join(modified_word)
                out.append(1)
                #chuyển word về dạng telex của nó, nếu nó ko có từ nào chuyển được thì giữ nguyên
            else:
                vn_letters = [char for char in token if char in dict1.keys()]
                if vn_letters:
                    # Randomly select one of the Vietnamese letters
                    modified_letter = random.choice(vn_letters)
                    # Replace the letter with its corresponding Telex coding
                    tokens[i] = tokens[i].replace(modified_letter, dict1[modified_letter])
                    out.append(1)
                else:
                    out.append(0)
        else:
            out.append(0)
        if count >= len(tokens) // 2:
            out.extend([0] * (len(tokens) - count))
            return ' '.join(tokens), out
    return ' '.join(tokens),out
def load_dataset(file_path):
    dataset = []
    with open(file_path, 'r',encoding= "utf-8") as f:
        for line in f:
            line = line.strip()
            dataset.append(line)
    return dataset
#check chạy ok
if __name__ == '_main_':
    dataset = load_dataset("text2.txt")
    error_dataset= [create_Error(i, 0.2) for i in dataset]
    df = pd.DataFrame(columns=['original','error','label'])
    df['original'] = dataset
    df['error'] = [x[0] for x in error_dataset]
    df['label'] = [x[1] for x in error_dataset]
    df = pd.DataFrame(columns=['original','error','label'], data=df)
    df.to_csv('process_data.csv', index=False)

In [4]:
dataset = ["Bình rất vui tính.", "Chuẩn là ngừoi thông minh."]
error_dataset= [create_Error(i, 1.1) for i in dataset]
df = pd.DataFrame(columns=['original','error','label'])
df['original'] = dataset
df['error'] = [x[0] for x in error_dataset]
df['label'] = [x[1] for x in error_dataset]
df = pd.DataFrame(columns=['original','error','label'], data=df)
df

,original,error,label
0,Bình rất vui tính.,Bì5h ất vui tính.,"[1, 1, 0, 0]"
1,Chuẩn là ngừoi thông minh.,Chuẩ4 law ngừoi thông minh.,"[1, 1, 0, 0, 0]"


In [3]:
import resource
import os
def save_sent(dataset, fileName):
    with open(fileName, 'w',encoding = 'utf-8') as f:
        for line in dataset:
            f.write(line+'\n')
    print('Done')

def save_mask(dataset, fileName):
    with open(fileName, 'w',encoding = 'utf-8') as f:
        for line in dataset:
            f.write(' '.join(str(e) for e in line)+'\n')
    print('Done')

def append_sent_to_file(file_path, dataset):
    with open(file_path, 'a',encoding = 'utf-8') as file:
        for line in dataset:
            file.write(line + '\n')

def append_mask_to_file(file_path, dataset):
    with open(file_path, 'a',encoding = 'utf-8') as file:
        for mask in dataset:
            file.write(' '.join(str(e) for e in mask)+'\n')

path = "/home/lenovo/Documents/1. FPT University/4.Project/Data/news-corpus-categorys-20181217/corpus/"
save_path_sent = "/home/lenovo/Documents/1. FPT University/4.Project/ProcessedData/sent/"
save_path_mask = "/home/lenovo/Documents/1. FPT University/4.Project/ProcessedData/mask/"
for file_name in os.listdir(path):
    print("Loading file:", file_name)
    file_name = os.path.join(path, file_name)
    error_dataset = []
    mask_dataset = []
    i = 0
    with open(file_name, encoding = 'utf-8') as txt_file:
        for line in txt_file:
            i = i + 1
            error_sent,mask = create_Error(line,0.2)
            error_dataset.append(error_sent)
            mask_dataset.append(mask)
            
            if i % 100 == 0:
                append_sent_to_file(os.path.join(save_path_sent, os.path.basename(file_name)), error_dataset)
                append_mask_to_file(os.path.join(save_path_mask, os.path.basename(file_name)), mask_dataset)
                # break
                error_dataset = []
                mask_dataset = []

        # check if there are any sent in the last lines
        if len(error_dataset) > 0:
            append_sent_to_file(os.path.join(save_path_sent, os.path.basename(file_name)), error_dataset)
            append_mask_to_file(os.path.join(save_path_mask, os.path.basename(file_name)), mask_dataset)


Loading file: Thế giới.txt


FileNotFoundError: [Errno 2] No such file or directory: '/home/lenovo/Documents/1. FPT University/4.Project/ProcessedData/sent/Thế giới.txt'

In [ ]:
append_sent_to_file("/home/lenovo/Documents/1. FPT University/4.Project/ProcessedData/Công nghệ sent.txt", error_dataset)
append_mask_to_file("/home/lenovo/Documents/1. FPT University/4.Project/ProcessedData/Công nghệ mask.txt", mask_dataset)

In [ ]:
error_dataset[26]

'VeriME - hệ `inh tháu Xác minh Nhận dạng số dựa trên nền tảng Blockchain vừa chính thức kys kết 6ợp tác với Avvanz - ông ty công nghệ Quản lý vòng ờđi nhân viên với giải pháp tìm kieesm n8ân sự cho ftổ chức doanh nghiệp, trong việc cung cấp dịch vụ Xác minh nhận dạng số (D-KYC).'